![](../images/rivacon_frontmark_combined_header.png)

# Day Counter, Roll Conventions and Schedules

In [ ]:
import pyvacon.analytics as analytics
import datetime
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import math
import pyvacon.marketdata.plot as mkt_plot #import module for plotting functionality
#the next line is a Jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline

## Schedules
For different financial instruments such as Bonds or Swaps a vector for certain dates is needed. These vectors are usually called schedules. They may describe the dates where certain payments or other events (such as fixings) occur. Such schedules are normally based on some construction logic such as "the first Monday of each month in year 2020". The generation of these schedules is therefore often not handmade by made by algorithms, which we call schedule generators. To create such a schedule with a generator, we have three main ingredients:
- Periods describing the frequency of the dates such as monthly, yearly, quarterly etc.
- A holiday calendar since most often schedules contain only business days
- A roll convention which defines what happens if the algorithm ends at a holiday
which we discuss in the following.

### Periods
A period is described by the number of years/months/days and is the basis in the schedule generation.

In [ ]:
period_1yr = analytics.Period(1,0,0) # create a period of 1 year (first argument of this method describes years, second month and last days)
period_3m = analytics.Period(0,3,0)
period_30days = analytics.Period(0,0,30)

### Holiday calendar
A holiday calender specifies which dates are holidays and which dates are business days. Here, we differ between two different types of holidays: Certain weekdays which are for the given location always holidays (such as Saturdays and Sundays in all western countries) and holidays belonging to certain dates. Therefore, the holiday calendar provides two different methods: One to configure a certain weekday as holiday and one to add a list of certain dates as holidays. 

In [ ]:
holidays = analytics.SimpleHolidayCalendar('GER_HOL')
holidays.setWeekdayAsHoliday(0) # set sunday as holiday
holidays.setWeekdayAsHoliday(6) #set saturday as holidays
holidays.setHolidays([analytics.ptime(2020,1,1,0,0,0),analytics.ptime(2021,1,1,0,0,0)])

### Schedule generation
To create a schedule we first have to create a specification containing the information described above (roll conventions, holidays, periods). We may then call the generate method to create a vector of dates defining the schedule. 

In [ ]:
start = analytics.ptime(2017,1,1,0,0,0)
end = analytics.ptime(2022,1,1,0,0,0)
schedule_spec_1yr_following = analytics.ScheduleSpecification(start,end, period_1yr, False, enums.RollConvention.FOLLOWING, holidays)
schedule_spec_1yr_preceding = analytics.ScheduleSpecification(start,end, period_1yr, False, enums.RollConvention.PRECEDING, holidays)
schedule_1yr_following = schedule_spec_1yr_following.generate()
print(converter.create_datetime_list(schedule_1yr_following))

schedule_1yr_preceding = schedule_spec_1yr_preceding.generate()
print(converter.create_datetime_list(schedule_1yr_preceding))

### Day counter
Day counters are used to compute accrued interest. There are several different day count conventions where we will present those currently implemented in the Analytics. A good overview on day count conventions as well other market conventions can be found [here](https://opengamma.com/wp-content/uploads/2017/11/Interest-Rate-Instruments-and-Market-Conventions.pdf).
The pyvacon library provides two interfaces to compute year fractions:
- the method yf with two dates as arguments computes the year fraction between the two given dates
- the method yf with three arguments: a vector of doubles containing the resulting year fractions, a reference date which which represents the first date the year fraction are computed with and a vector of second dates

#### 30/360 conventions
All conventions in this class assume that each month has 30 days and that the year has 360 days. 
The general formula is therefore given by 

$$ \frac{360 (Y_{2,adj} - Y_{1,adj}) + 30(M_{2,adj}-M_{1,adj}) + D_{2,adj}-D_{1,adj}}{ 360}$$ 

where $Y_{i,adj}$ denotes the adjusted year of date $i$, $M_{i,adj}$ the month and $D_{i,adj}$ the day. 
The methods differ on how they adjust the given dates as described below.
##### 30U/360
Adjustment rules are
- $D_1=31$ then set $D_1=30$
- $D_2=31$ and $D_1=30$ or $D_1=31$ then set $D_2=30$

This convention is also called 30/360 US, 30U/360 or 360/360.


In [ ]:
refdate = analytics.ptime(2017,1,1,0,0,0)
dc_30360 = analytics.DayCounter(enums.DayCounter.ThirtyU360)
yf = analytics.vectorDouble()
dc_30360.yf(yf, refdate, schedule_1yr_following)
for x in yf:
    print(x)

##### 30E/360
Adjustment rules are
- $D_1=31$ then set $D_1=30$
- $D_2=31$ then set $D_2=30$
which is also called Eurobond basis.

In [ ]:
refdate = analytics.ptime(2017,1,1,0,0,0)
dc_30360 = analytics.DayCounter(enums.DayCounter.ThirtyE360)
yf = analytics.vectorDouble()
dc_30360.yf(yf, refdate, schedule_1yr_following)
for x in yf:
    print(x)

#### ACT/365 Fixed
Here, the definition is 
$$ \frac{d_2-d_1}{365}$$
where $d_2-d_1$ is the number of dates between the two dates.

In [ ]:
refdate = analytics.ptime(2017,1,1,0,0,0)
dc_act365fixed = analytics.DayCounter(enums.DayCounter.ACT365_FIXED)
yf = analytics.vectorDouble()
dc_act365fixed.yf(yf, refdate, schedule_1yr_following)
for x in yf:
    print(x)

#### ACT/ACT
The year fraction is computed as  $$ \frac{Days\; in\; non leap\; year}{365} + \frac{Days\; in\; leap\; year}{366}$$

In [ ]:
refdate = analytics.ptime(2017,1,1,0,0,0)
dc_actact = analytics.DayCounter(enums.DayCounter.ACTACT)
yf = analytics.vectorDouble()
dc_actact.yf(yf, refdate, schedule_1yr_following)
for x in yf:
    print(x)

---